In [140]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [141]:
master = pd.read_pickle("nysenate_master")
doc_topics = pd.read_pickle("doc_topics_v2")

In [142]:
print(f"master shape: {master.shape}")
print(f"doc topics shape: {doc_topics.shape}")

master = master.drop_duplicates(['basePrintNoStr','publishedDateTime'])
# doc_topics = doc_topics.drop_duplicates()

print(f"master shape: {master.shape}")
print(f"doc topics shape: {doc_topics.shape}")

master shape: (133103, 215)
doc topics shape: (137701, 41)
master shape: (133098, 215)
doc topics shape: (137701, 41)


In [143]:
master.sample()

,_id,activeVersion,adopted,approvalMessage,basePrintNo,basePrintNoStr,printNo,publishedDateTime,session_x,signed,...,replacedBy,num_amendments,previousVersion,sponsor_chamber,sponsor_district,sponsor_fullName,sponsor_shortName,max_topic,topic_desc,month
32423,5b1c14225a67a40cd3484fa8,,False,None,A1976,A1976-2011,A1976,2011-01-12 16:00:22,2011,False,...,not_replaced,1,no_previousVersion,ASSEMBLY,52.0,Joan Millman,MILLMAN,9,"education, school districts",1


In [144]:
doc_topics.sample()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,max_topic
A5127-2017,0.000307,0.000649,0.0,0.000859,0.0,0.0017,0.0,0.0,0.036001,0.0,...,0.014383,0.000013,0.0,0.0,0.0,0.001522,0.0,0.0,0.0,8


In [145]:
bn_topic_descs ={0: 'utilities',
 1: 'businesses/employees',
 2: 'scouts',
 3: 'honoring resolutions',
 4: 'property taxes',
 5: 'criminal law',
 6: 'education',
 7: 'taxes',
 8: 'health care',
 9: 'honoring resolutions',
 10: 'retirement systems',
 11: 'honoring firefighters',
 12: 'county taxes',
 13: 'scouts',
 14: 'scouts',
 15: 'elections',
 16: 'high school athletics honors',
 17: 'criminal law',
 18: 'insurance',
 19: 'environmental',
 20: 'housing policy',
 21: 'honoring resolutions',
 22: 'motor vehicles',
 23: 'child services',
 24: 'motor vehicles',
 25: 'honoring resolutions',
 26: 'misc',
 27: 'sexual offenders',
 28: 'taxes',
 29: 'alcohol',
 30: 'public financing',
 31: 'amendments',
 32: 'highways',
 33: 'fuel standards',
 34: 'public commissions',
 35: 'education',
 36: 'health care',
 37: 'charities',
 38: 'NY constitution',
 39: 'animals'}

In [109]:
master[master['basePrintNoStr']=="A4482-2009"]
# master[master['_id']=="5b1c11ca5a67a40cd347d467"]
# master.dtypes

,_id,activeVersion,adopted,approvalMessage,basePrintNo,basePrintNoStr,printNo,publishedDateTime,session_x,signed,...,replacedBy,num_amendments,previousVersion,sponsor_chamber,sponsor_district,sponsor_fullName,sponsor_shortName,max_topic,topic_desc,month
1000,5b1c11cb5a67a40cd347d4e9,B,False,None,A4482,A4482-2009,A4482B,2009-01-01 00:00:01,2009,False,...,not_replaced,3,A4578-2007,ASSEMBLY,72.0,Adriano Espaillat,ESPAILLAT,A4482-2009 0.0 A4482-2009 0.0 A4482-2009...,no_topic_desc,2


In [147]:
def update_maxtopic(row):
    bpns = row['basePrintNoStr']
    new_max_topic = doc_topics.loc[bpns]['max_topic']
#     print(row['max_topic'])
#     print(row['topic_desc'])
#     print(row['basePrintNoStr'])
    
    row['max_topic'] = new_max_topic
#     print(new_max_topic)
#     print(row['_id'])
    try:
        row['topic_desc'] = bn_topic_descs[new_max_topic]
    except:
        row['topic_desc'] = 'manual_topic_input'
    
#     print("*"*50)
    return row

# master = master.apply(update_maxtopic,axis=1)
master = master.apply(update_maxtopic,axis=1)

Need to review topics 3,6,26 . 

In [124]:
# counter = 0
# def update_topicdesc(row):
#     mx_top = row['max_topic']
#     try:
#         row['topic_desc'] = bn_topic_descs[mx_top]
#     except:
#         row['topic_desc'] = 'no_topic_desc'
# #         counter +=1
#     return row
# master = master.apply(update_topicdesc,axis=1)

In [ ]:
# Bill: A4482-2009
# Doc Topic: 0.0
# **************************************************
# Bill: A405-2011
# Doc Topic: 20.0
# **************************************************
# Bill: S829-2013
# Doc Topic: 39.0
# **************************************************
# Bill: A456-2015
# Doc Topic: 30.0
# **************************************************
# Bill: S778-2017
# Doc Topic: 20.0
# **************************************************

In [151]:
update_dict={}
for idx, row in master.iterrows():
    if row['topic_desc'] == 'manual_topic_input':
        update_dict[row['basePrintNoStr']] = [idx]
        print(f"Bill: {row['basePrintNoStr']} | idx: {idx}")
print("-"*50)
for nb in master[master['topic_desc']=="manual_topic_input"]['basePrintNoStr'].values:
    print(f"Bill: {nb}")
    print(f"Doc Topic: {doc_topics.loc[nb]['max_topic'].min()}")
    update_dict[nb].append(doc_topics.loc[nb]['max_topic'].min())
    print("*"*50)
    
for bill, idx_topic in update_dict.items():
    master.loc[idx_topic[0],'max_topic'] = idx_topic[1]
    master.loc[idx_topic[0],'topic_desc'] = bn_topic_descs[idx_topic[1]]
    print(master.loc[idx_topic[0]])

Bill: A4482-2009 | idx: 1000
Bill: A405-2011 | idx: 29496
Bill: S829-2013 | idx: 55242
Bill: A456-2015 | idx: 80814
Bill: S778-2017 | idx: 107427
--------------------------------------------------
Bill: A4482-2009
Doc Topic: 0
**************************************************
Bill: A405-2011
Doc Topic: 36
**************************************************
Bill: S829-2013
Doc Topic: 37
**************************************************
Bill: A456-2015
Doc Topic: 33
**************************************************
Bill: S778-2017
Doc Topic: 18
**************************************************


In [167]:
pd.to_pickle(master,"nysenate_master_new")